# Using side information in retrieval

In the basic retrieval tutorial we built a simple retrieval model for the Movielens dataset, using user and movie ids as only features.

However, it's often useful to use a richer set of features, for both queries and candidates. For example:

1. In a large candidate catalogue, there may not be enough data per item to accurately estimate an embedding vector for every item. Using items features (categories, descriptions, images) will help build a model that's more accurate and generalizes better to unseen items.
2. Some applications (like fashion) have candidate sets that change frequently, and any given item is available only for a short time. This leaves little time for learning of accurate item representations.
3. Users' preferences may change depending on the context they are in. For example, a single user may prefer to consume short-form content when on their phone, and reserve long-form content for their TV. Being able to use the context of the interaction in the model will help capture these nuances.
4. Items' relevance may change over time. Including time as an explicit feature will help the model capture popularity trends, preventing items that were once popular but not relevant any more from dominating future recommendations.

In this example, we're going to make use of query context to improve our initial model.

## Preliminaries

As before, let's start with the necessary imports.

In [0]:
import os
import pprint
import tempfile

import numpy as np
import tensorflow as  tf
import tensorflow_datasets as tfds

import tensorflow_recommenders as tfrs

We also need to repeat the training/test splits and vocabulary building.

In [0]:
ratings = tfds.load("movie_lens/100k-ratings", split="train")
movies = tfds.load("movie_lens/100k-movies", split="train")

ratings = ratings.map(lambda x: {
    "movie_id": x["movie_id"],
    "user_id": x["user_id"],
    "timestamp": x["timestamp"],
})
movies = movies.map(lambda x: {
    "movie_id": x["movie_id"],
})

# Randomly shuffle data and split between train and test.
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

movie_ids = movies.batch(1_000_000).map(lambda x: x["movie_id"])
user_ids = ratings.batch(1_000_000).map(lambda x: x["user_id"])

unique_movie_ids = np.unique(np.concatenate(list(movie_ids)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

## Using query context information

The first piece of side information we can make use of is the time when a rating is given. This will help us capture two things:

1. Popularity dynamics: some movies are watched a lot when they are first released, but do not become classics. This makes them a good recommendation soon after release, but a bad recommendation later on.
2. User tastes changing over time. Making sure our model has the capacity to express this via interactions of user embeddings and time will help us capture this.

### Representing time

Our ratings dataset has raw timestamp features:

In [0]:
for row in ratings.take(2).as_numpy_iterator():
  print(f"Timestamp: {row['timestamp']}")

Timestamp: 879024327
Timestamp: 875654590


We can't use this directly: we need normalized features in order to make our learning algorithm stable.

There are many ways of doing this:
1. We could treat time as a continuous feature, and scale it to lie between 0 and 1. This coule be done via quantile scaling (like in sklearn's [QuantileTransformer](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.QuantileTransformer.html)) or through any variety of other methods that map an arbitrary real variable to a fixed interval.
2. We could treat time as a discrete variable: a series of discrete time periods.

The advantage of treating time as a continuous variable is that its effects become easy to estimate: there are only a few parameters added to the model. However, given enough data, treating time as discrete gives us a more flexible model, where each period as different effects, rather than following a smooth curve over time.

In this example, we're going to adopt the discrete approach. We'll start by dividing time into 1000 equal buckets.

In [0]:
max_timestamp = ratings.map(lambda x: x["timestamp"]).reduce(tf.cast(0, tf.int64), tf.maximum).numpy().max()
min_timestamp = ratings.map(lambda x: x["timestamp"]).reduce(np.int64(1e9), tf.minimum).numpy().min()

timestamp_buckets = np.linspace(min_timestamp, max_timestamp, num=1000)

print(f"Buckets: {timestamp_buckets[:3]}")

Buckets: [8.74724710e+08 8.74743291e+08 8.74761871e+08]


### Query model

We can now map those into embeddings via definig appropriate feature columns for our query model.

In [0]:
embedding_dimension = 32

#  An embedding column for user ids, as before.
user_id_feature = tf.feature_column.embedding_column(
        tf.feature_column.categorical_column_with_vocabulary_list(
            "user_id", unique_user_ids),
        embedding_dimension)

# An embedding column for the bucketized timestamps: there will be a separate
# embedding for each of the timestamp buckets.
time_feature = tf.feature_column.embedding_column(
        tf.feature_column.bucketized_column(
            tf.feature_column.numeric_column("timestamp"),
            timestamp_buckets.tolist()),
        embedding_dimension)

We can now use these features in the model. While we could use complex models here, let's start with something simple: we simply concatenate the time and user embedding, and project them onto the item embedding dimension (remember, the output dimensionality between the user model and the candidate model has to be the same).

In [0]:
user_model_input = {
    "user_id": tf.keras.Input(shape=(), dtype=tf.string, name="user_id"),
    "timestamp": tf.keras.Input(shape=(), dtype=tf.int64, name="timestamp"),
}
user_features = tf.keras.layers.DenseFeatures([user_id_feature, time_feature])(user_model_input)
user_embedding = tf.keras.layers.Dense(embedding_dimension)(user_features)
user_model = tf.keras.Model(user_model_input, user_embedding, name="user_model")

### Candidate model

Let's keep the candidate model as before - using id information only - and see what effect the inclusion of context information has on the model's accuracy.

In [0]:
movie_features = [tf.feature_column.embedding_column(
  tf.feature_column.categorical_column_with_vocabulary_list(
    "movie_id", list(unique_movie_ids)),
  embedding_dimension)]

movie_model_input = {"movie_id": tf.keras.Input(shape=(), dtype=tf.string)}
movie_embedding = tf.keras.layers.DenseFeatures(movie_features)(movie_model_input)
movie_model = tf.keras.Model(movie_model_input, movie_embedding, name="movie_model")

### Putting it all together

As before, we put it all together in a model class we'll use for fitting and evaluation.

In [0]:
class MovielensModel(tfrs.models.Model):

  def __init__(self):
    super().__init__()

    self.user_model: tf.keras.Model = user_model
    self.movie_model: tf.keras.Model = movie_model
    self.task = tfrs.tasks.RetrievalTask(
      corpus_metrics=tfrs.metrics.FactorizedTopK(
        candidates=movies.batch(128).map(movie_model)
      )
    )

  def compute_loss(self, features, training=False):

    user_embeddings = self.user_model({"user_id": features["user_id"],
                                       "timestamp": features["timestamp"]})
    positive_movie_embeddings = self.movie_model(
        {"movie_id": features["movie_id"]})

    return self.task(user_embeddings, positive_movie_embeddings)

In [0]:
model = MovielensModel()
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

Then shuffle, batch, and cache the training and evaluation data.

In [0]:
cached_train = train.shuffle(100_000).batch(8192).cache()
cached_test = test.batch(4096).cache()

Then train the  model:

In [0]:
model.fit(cached_train, epochs=3)

Epoch 1/3
10/10 [==============================] - 14s 1s/step - factorized_top_k: 0.0348 - factorized_top_k/top_1_categorical_accuracy: 8.8750e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0051 - factorized_top_k/top_10_categorical_accuracy: 0.0108 - factorized_top_k/top_50_categorical_accuracy: 0.0534 - factorized_top_k/top_100_categorical_accuracy: 0.1039 - loss: 69728.0987
Epoch 2/3
10/10 [==============================] - 15s 1s/step - factorized_top_k: 0.0791 - factorized_top_k/top_1_categorical_accuracy: 0.0018 - factorized_top_k/top_5_categorical_accuracy: 0.0122 - factorized_top_k/top_10_categorical_accuracy: 0.0258 - factorized_top_k/top_50_categorical_accuracy: 0.1235 - factorized_top_k/top_100_categorical_accuracy: 0.2324 - loss: 67110.0483
Epoch 3/3
10/10 [==============================] - 16s 2s/step - factorized_top_k: 0.1049 - factorized_top_k/top_1_categorical_accuracy: 0.0022 - factorized_top_k/top_5_categorical_accuracy: 0.0188 - factorized_top_k/top_10_catego

### Results

What do the results look like?

In [0]:
model.evaluate(cached_test, return_dict=True)

5/5 [==============================] - 3s 526ms/step - factorized_top_k: 0.0721 - factorized_top_k/top_1_categorical_accuracy: 7.0000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0076 - factorized_top_k/top_10_categorical_accuracy: 0.0163 - factorized_top_k/top_50_categorical_accuracy: 0.1137 - factorized_top_k/top_100_categorical_accuracy: 0.2222 - loss: 31004.3128


{'factorized_top_k': array([0.0007 , 0.00755, 0.0163 , 0.1137 , 0.2222 ], dtype=float32),
 'factorized_top_k/top_1_categorical_accuracy': 0.000699999975040555,
 'factorized_top_k/top_5_categorical_accuracy': 0.007550000213086605,
 'factorized_top_k/top_10_categorical_accuracy': 0.016300000250339508,
 'factorized_top_k/top_50_categorical_accuracy': 0.1137000024318695,
 'factorized_top_k/top_100_categorical_accuracy': 0.22220000624656677,
 'loss': 28181.4375}

Our accuracies for all values of k are quite a bit better than the model from the basic retrieval tutorial. Clearly, accounting for time is useful.

When using timestamp information, the performance of the model can vary with the number of buckets we use when bucketizing the timestamps. In this tutorial we used 1000 buckets, which significantly improved the performance of the model. However, when using 100 buckets, the performance of the model using timestamp information would not be significantly better than the performance of the model from the basic retrieval tutorial.